# Import modules and set core NRPy+ parameters

In [1]:
from outputC import lhrh, outCfunction, outC_function_dict
import finite_difference as fin
import NRPy_param_funcs as par
import grid as gri
import indexedexp as ixp
import reference_metric as rfm
import cmdline_helper as cmd
import shutil, os, sys, time
import pickle

In [2]:
# Create C code output directory
Ccodesdir = os.path.join("BSSN_LinearScalarFieldEvolution_Ccodes/")
# Remove C code output directory if it exists
shutil.rmtree(Ccodesdir, ignore_errors=True)
# Create a fresh directory
cmd.mkdir(Ccodesdir)

# Create executable output directory
outdir = os.path.join(Ccodesdir, "output/")
cmd.mkdir(outdir)

In [3]:
# Set spatial dimension to 3
par.set_parval_from_str("grid::DIM", 3)
DIM = par.parval_from_str("grid::DIM")

# Enable/Disable SIMD-optimised code
SIMD_enable = True

# If SIMD is enabled, copy SIMD_intrinsics.h to the C codes directory
cmd.mkdir(os.path.join(Ccodesdir, "SIMD/"))
if SIMD_enable == True:
    shutil.copy(os.path.join("SIMD/", "SIMD_intrinsics.h"), os.path.join(Ccodesdir, "SIMD/"))
else:
    with open(os.path.join(Ccodesdir, "SIMD", "SIMD_intrinsics.h"), "w") as file:
        file.write("#define SIMD_IS_DISABLED\n")

# Set coordinate system and call the reference metric
CoordSystem = "Spherical"
par.set_parval_from_str("reference_metric::CoordSystem", CoordSystem)
rfm.reference_metric()

# Set domain size
domain_size = 10

# Set sinh width (for Sinh* coordinates)
sinh_width = 0.4

# Set finite differencing order
FD_order = 4
par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", FD_order)

# Allow for the use of functions (instead of inline calculations) to compute finite differences
FD_functions_enable = True
par.set_parval_from_str("finite_difference::FD_functions_enable", FD_functions_enable)

# Set the core data type and the default CFL factor
REAL = "double"
default_CFL_FACTOR = 0.5

# Set mass and spin of the Black Hole
M = 1
chi = 0.99

In [4]:
# Generate Runge-Kutta timestepping code
import MoLtimestepping.C_Code_Generation as MoL
from MoLtimestepping.RK_Butcher_Table_Dictionary import Butcher_dict

RK_method = "RK4"
RK_order = Butcher_dict[RK_method][1]
cmd.mkdir(os.path.join(Ccodesdir, "MoLtimestepping/"))
MoL.MoL_C_Code_Generation(RK_method,
    RHS_string="""
Ricci_eval(&rfmstruct, &params, RK_INPUT_GFS, auxevol_gfs);
rhs_eval(&rfmstruct, &params, auxevol_gfs, RK_INPUT_GFS, RK_OUTPUT_GFS);""",
    post_RHS_string = """
apply_bcs_curvilinear(&params, &bcstruct, NUM_EVOL_GFS, evol_gf_parity, RK_OUTPUT_GFS);
enforce_detgammabar_constraint(&rfmstruct, &params,                     RK_OUTPUT_GFS);\n""",
    outdir = os.path.join(Ccodesdir,"MoLtimestepping/"))

In [5]:
# Output the find_timestep() function to a C file
rfm.out_timestep_func_to_file(os.path.join(Ccodesdir, "find_timestep.h"))

In [6]:
# In the parallel C codegen below, the
def pickled_outC_function_dict(outC_function_dict):
    outstr = []
    outstr.append(pickle.dumps(len(outC_function_dict)))
    for Cfuncname, Cfunc in outC_function_dict.items():
        outstr.append(pickle.dumps(Cfuncname))
        outstr.append(pickle.dumps(Cfunc))
    return outstr

# Import initial data for the UIUC Black Hole

In [7]:
import BSSN.UIUCBlackHole as uiuc

def UIUCBlackHoleID():
    print("Generating optimised C code for UIUC Black Hole initial data. May take a while, depending on CoordSystem.")
    start = time.time()

    uiuc.UIUCBlackHole()    # Registers ID C function in dictionary, used below to output to file
    with open(os.path.join(Ccodesdir, "initial_data.h"), "w") as file:
        file.write(outC_function_dict["initial_data"])
    end = time.time()
    print(f"(BENCH) Finished UIUC BH initial data codegen in {end - start} seconds.")
    return pickled_outC_function_dict(outC_function_dict)

# Output C Code for BSSN evolution of the spacetime

In [8]:
import BSSN.BSSN_RHSs as rhs
import BSSN.BSSN_gauge_RHSs as grhs

# Set the *covariant*, second-order Gamma-driving shift condition
par.set_parval_from_str("BSSN.BSSN_gauge_RHSs::ShiftEvolutionOption", "GammaDriving2ndOrder_Covariant")

print("Generating symbolic expressions for BSSN RHSs...")
start = time.time()

# Enable rfm_precompute to eliminate transcendental functions from BSSN RHSs
cmd.mkdir(os.path.join(Ccodesdir, "rfm_files/"))
par.set_parval_from_str("reference_metric::enable_rfm_precompute", "True")
par.set_parval_from_str("reference_metric::rfm_precompute_Ccode_outdir", os.path.join(Ccodesdir, "rfm_files/"))

# Evaluate BSSN + BSSN gauge RHSs with rfm_precompute enabled
import BSSN.BSSN_quantities as Bq
# Setting LeaveRicciSymbolic to True leads to a significantly shorter RHS symbolic generation,
# at the small cost of a two-step MoL algorithm to compute RHSs in the C code
par.set_parval_from_str("BSSN.BSSN_quantities::LeaveRicciSymbolic", "True")

rhs.BSSN_RHSs()
grhs.BSSN_gauge_RHSs()

# Use betaU as the upwinding control vector
Bq.BSSN_basic_tensors()
betaU = Bq.betaU

# Generate symbolic expressions to enforce the constraint on the determinant of gammabar
import BSSN.Enforce_Detgammabar_Constraint as EGC
enforce_detg_constraint_symb_expressions = EGC.Enforce_Detgammabar_Constraint_symb_expressions()

# Compute the Ricci tensor
par.set_parval_from_str("BSSN.BSSN_quantities::LeaveRicciSymbolic", "False")
Bq.RicciBar__gammabarDD_dHatD__DGammaUDD__DGammaU()

# Register the Hamiltonian constraint as a grid function
H = gri.register_gridfunctions("AUX", "H")
# Define the Hamiltonian constraint and ouptut the optimised C code
import BSSN.BSSN_constraints as bssncon
bssncon.BSSN_constraints(add_T4UUmunu_source_terms=False)

# Restore all rfm hatted quantities to their close-form expressions by setting rfm_precompute to False
par.set_parval_from_str("reference_metric::enable_rfm_precompute", "False")
rfm.ref_metric__hatted_quantities()

# Print elapsed time
end = time.time()
print(f"(BENCH) Finished BSSN symbolic expression generation in {end - start} seconds.")

includes = None
if FD_functions_enable:
    includes = ["finite_difference_functions.h"]

Generating symbolic expressions for BSSN RHSs...
(BENCH) Finished BSSN symbolic expression generation in 5.343959808349609 seconds.


In [9]:
# Function to output C code for computing the BSSN right-hand sides
def BSSN_RHSs():
    print(f"Generating C code for BSSN RHSs in {par.parval_from_str('reference_metric::CoordSystem')} coordinates.")
    start = time.time()

    # Construct the left-hand sides and right-hand side expressions for all BSSN RHSs
    lhs_names = [    "alpha",       "cf",       "trK"]
    rhs_exprs = [grhs.alpha_rhs, rhs.cf_rhs, rhs.trK_rhs]
    for i in range(3):
        lhs_names.append(   f"betU{i}")
        rhs_exprs.append(grhs.bet_rhsU[i])
        lhs_names.append(  f"lambdaU{i}")
        rhs_exprs.append(rhs.lambda_rhsU[i])
        lhs_names.append(   f"vetU{i}")
        rhs_exprs.append(grhs.vet_rhsU[i])
        for j in range(i, 3):
            lhs_names.append(f"aDD{i}{j}")
            rhs_exprs.append(rhs.a_rhsDD[i][j])
            lhs_names.append(f"hDD{i}{j}")
            rhs_exprs.append(rhs.h_rhsDD[i][j])

    # Sort the lhss list alphabetically, and the rhss to match
    lhs_names, rhs_exprs = [list(x) for x in zip(*sorted(zip(lhs_names, rhs_exprs), key=lambda pair: pair[0]))]

    # Declare the list of lhrh's
    BSSN_evol_rhss = []
    for var in range(len(lhs_names)):
        BSSN_evol_rhss.append(lhrh(lhs=gri.gfaccess("rhs_gfs", lhs_names[var]), rhs=rhs_exprs[var]))

    # Set up the C function for the BSSN RHSs
    # Set up the outputC and loop parameters for BSSN_RHSs C function
    outC_params = "outCverbose=False"
    loopoptions = "InteriorPoints,Enable_rfm_precompute"
    if SIMD_enable == True:
        loopoptions += ",EnableSIMD"
        outC_params += ",SIMD_enable=True"
    desc = "Evaluate the BSSN RHSs"
    name = "rhs_eval"
    outCfunction(
        outfile = os.path.join(Ccodesdir, name + ".h"), includes=includes, desc=desc, name=name,
        params = """rfm_struct *restrict rfmstruct, const paramstruct *restrict params,
                    const REAL *restrict auxevol_gfs, const REAL *restrict in_gfs, REAL *restrict rhs_gfs""",
        body = fin.FD_outputC("returnstring", BSSN_evol_rhss, params=outC_params, upwindcontrolvec=betaU).replace("IDX4", "IDX4S"),
        loopopts=loopoptions
    )
    
    # Print elapsed time
    end = time.time()
    print(f"(BENCH) Finished BSSN_RHS C codegen in {end - start} seconds.")
    return pickled_outC_function_dict(outC_function_dict)

In [10]:
# Function to output C code for computing the Ricci tensor
def Ricci():
    print(f"Generating C code for Ricci tensor in {par.parval_from_str('reference_metric::CoordSystem')} coordinates.")
    start = time.time()

    # Set up the C function for the Ricci tensor
    # Set outputC and loop parameters for Ricci tensor function
    outC_params = "outCverbose=False"
    loopoptions = "InteriorPoints,Enable_rfm_precompute"
    if SIMD_enable == True:
        loopoptions += ",Enable_SIMD"
        outC_params += ",SIMD_enable=True"
    desc = "Evaluate the Ricci tensor"
    name = "Ricci_eval"
    outCfunction(
        outfile = os.path.join(Ccodesdir, name+".h"), includes=includes, desc=desc, name=name,
        params = """rfm_struct *restrict rfmstruct, const paramstruct *restrict params,
                    const REAL *restrict in_gfs, REAL *restrict auxevol_gfs""",
        body = fin.FD_outputC("returnstring",
                              [lhrh(lhs=gri.gfaccess("auxevol_gfs","RbarDD00"),rhs=Bq.RbarDD[0][0]),
                               lhrh(lhs=gri.gfaccess("auxevol_gfs","RbarDD01"),rhs=Bq.RbarDD[0][1]),
                               lhrh(lhs=gri.gfaccess("auxevol_gfs","RbarDD02"),rhs=Bq.RbarDD[0][2]),
                               lhrh(lhs=gri.gfaccess("auxevol_gfs","RbarDD11"),rhs=Bq.RbarDD[1][1]),
                               lhrh(lhs=gri.gfaccess("auxevol_gfs","RbarDD12"),rhs=Bq.RbarDD[1][2]),
                               lhrh(lhs=gri.gfaccess("auxevol_gfs","RbarDD22"),rhs=Bq.RbarDD[2][2])],
                               params=outC_params.replace("IDX4", "IDX4S")),
        loopopts=loopoptions
    )

    # Print elapsed time
    end = time.time()
    print(f"(BENCH) Finished Ricci C codegen in {end - start} seconds.")
    return pickled_outC_function_dict(outC_function_dict)

In [11]:
# Function to output C code for computing the Hamiltonian constraint
def Hamiltonian():
    print("Generating optimised C code for the Hamiltonian constraint. May take a while, depending on CoordSystem.")
    start = time.time()

    # Set up the C function for the Hamiltonian RHS
    desc = "Evaluate the Hamiltonian constraint"
    name = "Hamiltonian_constraint"
    outCfunction(
        outfile = os.path.join(Ccodesdir, name + ".h"), includes=includes, desc=desc, name=name,
        params = """rfm_struct *restrict rfmstruct, const paramstruct *restrict params,
                    REAL *restrict in_gfs, REAL *restrict aux_gfs""",
        body = fin.FD_outputC("returnstring", lhrh(lhs=gri.gfaccess("aux_gfs", "H"), rhs=bssncon.H),
                               params="outCverbose=False").replace("IDX4", "IDX4S"),
        loopopts="InteriorPoints,Enable_rfm_precompute"
    )

    # Print elapsed time
    end = time.time()
    print(f"(BENCH) Finished Hamiltonian C codegen in {end - start} seconds.")
    return pickled_outC_function_dict(outC_function_dict)

In [12]:
# Function to output C code for enforcing the constraint on the determinant of gammabar
def gammadet():
    print("Generating optimised C code for gamma gonstraint. May take a while, depending on CoordSystem.")
    start = time.time()

    # Set up the C function for the det(gammahat) = det(gammabar)
    EGC.output_Enforce_Detgammabar_Constraint_Ccode(Ccodesdir, exprs=enforce_detg_constraint_symb_expressions)
    
    # Print elapsed time
    end = time.time()
    print(f"(BENCH) Finished gamma constraint C codegen in {end - start} seconds.")
    return pickled_outC_function_dict(outC_function_dict)

In [13]:
# Generate C code kernels for BSSN expressions, in parallel if possible

# Create a list of functions we wish to evaluate in parallel (if possible)
funcs = [UIUCBlackHoleID, BSSN_RHSs, Ricci, Hamiltonian, gammadet]
# pickled_outC_func_dict stores outC_function_dict from all the subprocesses in the parallel codegen
pickled_outC_func_dict = []

try:
    if os.name == 'nt':
        # It's a mess to get working in Windows, so we don't bother. :/
        #  https://medium.com/@grvsinghal/speed-up-your-python-code-using-multiprocessing-on-windows-and-jupyter-or-ipython-2714b49d6fac
        raise Exception("Parallel codegen currently not available in certain environments, e.g., Windows")

    # Import the multiprocessing module.
    import multiprocessing as mp

    # Define master functions for parallelisation.
    def master_func(arg):
        return funcs[arg]()
    # Evaluate list of functions in parallel if possible
    # otherwise fallback to serial evaluation
    pool = mp.Pool()
    pickled_outC_func_dict.append(pool.map(master_func,range(len(funcs))))
    pool.terminate()
    pool.join()
except:
    # Fallback to serial evaluation if parallel not possible
    for func in funcs:
        func()
    pickled_outC_func_dict = []

# Output functions for computing all finite-difference stencils
if FD_functions_enable and len(pickled_outC_func_dict)>0:
    # First unpickle pickled_outC_func_dict
    outCfunc_dict = {}
    for WhichFunc in pickled_outC_func_dict[0]:
        i = 0
        num_elements = pickle.loads(WhichFunc[i]); i += 1
        for lst in range(num_elements):
            funcname = pickle.loads(WhichFunc[i])
            funcbody = pickle.loads(WhichFunc[i + 1]); i += 2
            outCfunc_dict[funcname] = funcbody
    # Then store the unpickled outCfunc_dict to outputC's outC_function_dict
    for key, item in outCfunc_dict.items():
        outC_function_dict[key] = item
        
if FD_functions_enable:
    # Finally generate finite_difference_functions.h
    fin.output_finite_difference_functions_h(path=Ccodesdir)

Generating C code for BSSN RHSs in Spherical coordinates.
Generating optimised C code for UIUC Black Hole initial data. May take a while, depending on CoordSystem.
Generating optimised C code for the Hamiltonian constraint. May take a while, depending on CoordSystem.
Generating optimised C code for gamma gonstraint. May take a while, depending on CoordSystem.
Generating C code for Ricci tensor in Spherical coordinates.
Output C function enforce_detgammabar_constraint() to file BSSN_LinearScalarFieldEvolution_Ccodes/enforce_detgammabar_constraint.h
(BENCH) Finished gamma constraint C codegen in 0.13461661338806152 seconds.
(BENCH) Finished UIUC BH initial data codegen in 1.0696113109588623 seconds.
Output C function rhs_eval() to file BSSN_LinearScalarFieldEvolution_Ccodes/rhs_eval.h
(BENCH) Finished BSSN_RHS C codegen in 13.475466966629028 seconds.
Output C function Ricci_eval() to file BSSN_LinearScalarFieldEvolution_Ccodes/Ricci_eval.h
(BENCH) Finished Ricci C codegen in 25.095694780

In [14]:
# Output C codes needed for declaring and setting Cparameters; also set free_parameters.h

# Generate declare_Cparameters_struct.h, set_Cparameters_default.h and set_Cparameters[-SIMD].h
par.generate_Cparameters_Ccodes(os.path.join(Ccodesdir))

# Set free_parameters.h
with open(os.path.join(Ccodesdir, "free_parameters.h"), "w") as file:
    file.write("""
// Set Black Hole mass and spin
params.M = {M};
params.chi = {chi};
    """)

# Append to $Ccodesdir/free_parameters.h reference metric parameters based on generic
# domain_size,sinh_width,sinhv2_const_dr,SymTP_bScale, parameters set above.
rfm.out_default_free_parameters_for_rfm(os.path.join(Ccodesdir,"free_parameters.h"), domain_size, sinh_width)

# Generate set_Nxx_dxx_invdx_params__and__xx.h
rfm.set_Nxx_dxx_invdx_params__and__xx_h(Ccodesdir)

# Generate xxCart.h to map from xx to cartesian coordinates
rfm.xxCart_h("xxCart", "./set_Cparameters.h", os.path.join(Ccodesdir, "xxCart.h"))

# Generate declare_Cparameters_struct.h, set_Cparameters_default.h and set_Cparameters[-SIMD].h
par.generate_Cparameters_Ccodes(os.path.join(Ccodesdir))

# Set up boundary conditions

In [15]:
import CurviBoundaryConditions.CurviBoundaryConditions as cbcs
cbcs.Set_up_CurviBoundaryConditions(os.path.join(Ccodesdir,"boundary_conditions/"),Cparamspath=os.path.join("../"))

Wrote to file "BSSN_LinearScalarFieldEvolution_Ccodes/boundary_conditions/parity_conditions_symbolic_dot_products.h"
Evolved parity: ( aDD00:4, aDD01:5, aDD02:6, aDD11:7, aDD12:8, aDD22:9,
    alpha:0, betU0:1, betU1:2, betU2:3, cf:0, hDD00:4, hDD01:5, hDD02:6,
    hDD11:7, hDD12:8, hDD22:9, lambdaU0:1, lambdaU1:2, lambdaU2:3, trK:0,
    vetU0:1, vetU1:2, vetU2:3 )
Auxiliary parity: ( H:0 )
AuxEvol parity: ( RbarDD00:4, RbarDD01:5, RbarDD02:6, RbarDD11:7,
    RbarDD12:8, RbarDD22:9 )
Wrote to file "BSSN_LinearScalarFieldEvolution_Ccodes/boundary_conditions/EigenCoord_Cart_to_xx.h"
